Example code from https://www.hackster.io/news/easy-tinyml-on-esp32-and-arduino-a9dbc509f26c

Guide for tinymlgen at https://github.com/eloquentarduino/tinymlgen

In [33]:
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tinymlgen import port

# def get_model():
#     SAMPLES = 1000
#     np.random.seed(1337)
#     x_values = np.random.uniform(low=0, high=2*math.pi, size=SAMPLES)
#     # shuffle and add noise
#     np.random.shuffle(x_values)
#     y_values = np.sin(x_values)
#     y_values += 0.1 * np.random.randn(*y_values.shape)

#     # split into train, validation, test
#     TRAIN_SPLIT =  int(0.6 * SAMPLES)
#     TEST_SPLIT = int(0.2 * SAMPLES + TRAIN_SPLIT)
#     x_train, x_test, x_validate = np.split(x_values, [TRAIN_SPLIT, TEST_SPLIT])
#     y_train, y_test, y_validate = np.split(y_values, [TRAIN_SPLIT, TEST_SPLIT])

#     # create a NN with 2 layers of 16 neurons
#     model = tf.keras.Sequential()
#     model.add(layers.Dense(16, activation='relu', input_shape=(1,)))
#     model.add(layers.Dense(16, activation='relu'))
#     model.add(layers.Dense(1))
#     model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
#     model.fit(x_train, y_train, epochs=200, batch_size=16,
#                         validation_data=(x_validate, y_validate))
#     return model

def get_model():
    SAMPLES = 1000
    np.random.seed(1337)
    x1_values = np.random.randint(low=1, high=51, size=SAMPLES)
    x2_values = np.random.randint(low=1, high=51, size=SAMPLES)
    y_values = x1_values.astype(np.float64) + x2_values.astype(np.float64)
    y_values += (0.01 * np.random.randn(*y_values.shape))

    # Reshape data
    x1_values = np.reshape(x1_values, (SAMPLES, 1))
    x2_values = np.reshape(x2_values, (SAMPLES, 1))
    y_values = np.reshape(y_values, (SAMPLES, 1))

    # Merge into 1 input tensor
    x_values = np.concatenate((x1_values, x2_values), axis=1)

    # split into train, validation, test
    TRAIN_SPLIT =  int(0.6 * SAMPLES)
    TEST_SPLIT = int(0.2 * SAMPLES + TRAIN_SPLIT)
    x_train, x_test, x_validate = np.split(x_values, [TRAIN_SPLIT, TEST_SPLIT])
    y_train, y_test, y_validate = np.split(y_values, [TRAIN_SPLIT, TEST_SPLIT])

    # Print the shapes of the new input tensors
    print(x_train.shape)
    print(y_train.shape)

    # create a NN with 2 layers of 16 neurons
    model = tf.keras.Sequential()
    model.add(layers.Dense(16, activation='relu', input_shape=(2,)))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    model.fit(x_train, y_train, epochs=200, batch_size=16,
                        validation_data=(x_validate, y_validate))
    return model

# model = get_model()
# c_code = port(model, pretty_print=True)
# print(c_code)

model = get_model()
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()
model.summary()

# Save the model to disk
open("multiplier_model_quantized.tflite", "wb").write(tflite_model)

(600, 2)
(600, 1)
Epoch 1/200
38/38 [==============================] - 1s 4ms/step - loss: 3329.2090 - mae: 53.4198 - val_loss: 2327.5583 - val_mae: 44.0532
Epoch 2/200
38/38 [==============================] - 0s 2ms/step - loss: 2004.3469 - mae: 41.2503 - val_loss: 1359.7195 - val_mae: 33.1759
Epoch 3/200
38/38 [==============================] - 0s 2ms/step - loss: 1058.1454 - mae: 29.3499 - val_loss: 636.2618 - val_mae: 21.5931
Epoch 4/200
38/38 [==============================] - 0s 2ms/step - loss: 443.8395 - mae: 17.6649 - val_loss: 245.8791 - val_mae: 12.6614
Epoch 5/200
38/38 [==============================] - 0s 2ms/step - loss: 149.0402 - mae: 9.9470 - val_loss: 94.1747 - val_mae: 7.9244
Epoch 6/200
38/38 [==============================] - 0s 2ms/step - loss: 61.3737 - mae: 6.4696 - val_loss: 43.8685 - val_mae: 5.4235
Epoch 7/200
38/38 [==============================] - 0s 2ms/step - loss: 24.8729 - mae: 4.0683 - val_loss: 12.3607 - val_mae: 2.8744
Epoch 8/200
38/38 [==========

INFO:tensorflow:Assets written to: C:\Users\jared\AppData\Local\Temp\tmp8z1vga7h\assets


INFO:tensorflow:Assets written to: C:\Users\jared\AppData\Local\Temp\tmp8z1vga7h\assets


Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_57 (Dense)            (None, 16)                48        
                                                                 
 dense_58 (Dense)            (None, 16)                272       
                                                                 
 dense_59 (Dense)            (None, 1)                 17        
                                                                 
Total params: 337 (1.32 KB)
Trainable params: 337 (1.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


3308

Converting of model to C array (Run below line on bash)

xxd -i multi_model_quantized.tflite > multi_model_quantized.cc